In [1]:
%matplotlib inline
import plot_init as pu
import track
import logging
import numpy as np
import matplotlib.pyplot as plt
logger = logging.getLogger()
logger.setLevel(30)

/home/ddailisan/.conda-envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ddailisan/Yolov5_StrongSORT_OSNet/strong_sort/deep/reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
source = "20220613T155958.mp4"
# source = "20220613T165013.mp4"

source_root = source.split('/')[-1].split('.mp4')[0]
weights = "yolov5/weights/crowdhuman_yolov5m.pt"
classes = 0
save_txt = True
save_vid = True
hide_conf = True
project='runs/track'
name = f'baguio_{source}'
line_thickness = 1
exist_ok = True
every_frame = 5
# track.run(
#         source=source,
#         yolo_weights=weights,  # model.pt path(s),
#         save_txt=save_txt,  # save results to *.txt
#         save_vid=save_vid,  # save confidences in --save-txt labels
#         classes=classes,  # filter by class: --class 0, or --class 0 2 3
#         line_thickness=line_thickness, 
#         name=name,
#         project=project,  # save results to project/name
#         exist_ok=exist_ok,
#         hide_conf=hide_conf,  # hide confidences
#         half=False,  # use FP16 half-precision inference
#         every_frame=every_frame, # skip frames for faster processing
# )

In [3]:
import pandas as pd


outputfile = project + '/' + name + '/tracks/' + source_root + '.txt'
col_names = ['frame_number', 'object_id', 'bbox_left', 'bbox_top', 'bbox_w', 'bbox_h', '1', '2', '3', 'class']
df = pd.read_csv(outputfile, delimiter=' ', names=col_names, index_col=False)

In [4]:
start_time = np.datetime64('2022-06-13T15:59:58.000')
# start_time = np.datetime64('2022-06-13T16:50:13.000')

In [5]:
fps = 30
dt = 1/fps

In [6]:
from scipy.signal import savgol_filter

counts = df.groupby('frame_number')[['object_id']].count().reset_index()
counts['time'] = counts['frame_number'].transform(lambda x: start_time+np.timedelta64(int(x*dt*1000),'ms'))
counts = counts.set_index('time')
counts['smoothed'] = savgol_filter(counts['object_id'], 60, 3) # window size 60, polynomial order 3

In [7]:


fig, ax = plt.subplots(1, figsize=pu.figsizes['single'])
counts.plot(y='object_id', color='0.3', linewidth=0.5, #linestyle='', 
            alpha=0.4,
            marker='o', markersize=2, legend=False, ax=ax)
counts.plot(y='smoothed', color='indianred', linewidth=1.2, label="smoothed 10 secs", ax=ax)

ax.set_ylabel('count')


fig.savefig(f'counts_{source_root}.pdf', bbox_inches='tight')

In [8]:
unique_id = df.groupby('frame_number')[['object_id']].max().reset_index()
unique_id['time'] = unique_id['frame_number'].transform(lambda x: start_time+np.timedelta64(int(x*dt*1000),'ms'))
unique_id = unique_id.set_index('time')


In [21]:
import scipy.stats as stats
import datetime

timestamp = unique_id.index.map(datetime.datetime.timestamp).values
timestamp -= timestamp[0] # in seconds
m, b, r, _,_, = stats.linregress(timestamp, unique_id.object_id)

fig, ax = plt.subplots(1, figsize=pu.figsizes['single'])
unique_id.plot(y='object_id', color='0.3', linewidth=0.5, #linestyle='', 
            alpha=0.4,
            marker='o', markersize=2, legend=False, ax=ax)
# unique_id.plot(y='smoothed', color='indianred', linewidth=1.2, label="smoothed 10 secs", ax=ax)

ax.set_ylabel('count')
ax.text(0.95,0.1, f'$y=({m:.2f}\\frac{{person}}{{sec}})x+{b:.2f}$', transform=ax.transAxes,
       verticalalignment='center', horizontalalignment='right', fontsize=10)

print(f'{m*3600:.2f} persons/hr')
fig.savefig(f'ids_{source_root}.pdf', bbox_inches='tight')

23910.28 persons/hr


In [10]:
date = datetime.date

In [11]:
unique_id.index.map(datetime.datetime.timestamp).values

array([ 1.6551e+09,  1.6551e+09,  1.6551e+09, ...,  1.6551e+09,  1.6551e+09,  1.6551e+09])

In [13]:
m*3600

23910.280966017403

In [19]:
unique_id.object_id.max()

6043